# PROYECTO FINAL

La empresa GoGreen Bikesharing, se dedica al alquiler de bicicletas y posee datos tales como la cantidad de bicis alquiladas por usuarios registrados, la cantidad de alquileres realizados por usuarios puntuales, y la cantidad total. 

A estos datos se les añadió información meteorológica, y el calendario de festivos.

Ahora buscan analizar cuáles son los aspectos que más influyen en la cantidad de bicis que van a alquilar en un día.

In [53]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sidetable
import datetime

PREGUNTAS

¿en que momento del año se alquilan mas bicicletas?
Influye soleado o nubes?
Fin de semana o laborable?
los primeros y ultimos dias del mes hay mas reservas?
festivos aumenta las reservas?
la tempreatura a partir de la cual disminuye los alquileres  ->Punto ruptura
Punto maximo de alquiler
la temperatura alta influye al alquiler de espontaneos o habituales?
la humedad es un punto clave en algun momento? A partir de que punto de humedad bajan drasticamente las reservas?

fecha donde hay mas y menos reservas






In [54]:
df_bike = pd.read_csv("./bikes.csv", index_col=0)
df_bike.sample(6)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
46,47,16-02-2018,spring,0,2,0,3,1,1,13.048701,15.81000,42.3478,16.869997,218,1897,2115
224,225,13-08-2018,autumn,0,8,0,6,0,2,28.119153,32.29290,72.9583,14.167418,1504,2646,4150
160,161,10-06-2018,summer,0,6,0,5,1,1,30.955000,35.16460,60.5000,9.417118,815,3771,4586
161,162,11-06-2018,summer,0,6,0,6,0,1,29.725000,33.90190,65.4583,10.374950,1729,3237,4966
325,326,22-11-2018,winter,0,11,0,2,1,3,17.083347,21.08480,96.2500,7.959064,69,1538,1607
122,123,03-05-2018,summer,0,5,0,2,1,2,25.283347,29.10395,69.7083,22.958689,603,3848,4451


In [5]:
df_bike.mnth.dtype

dtype('int64')

# limpieza string

In [10]:
df_bike.season.unique()

array(['spring', 'summer', 'autumn', 'winter'], dtype=object)

# duplicados

In [3]:
df_bike.duplicated().sum()

0

# Nulos

In [4]:
df_bike.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [7]:
df_bike.shape

(730, 16)

In [6]:
df_bike.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    object 
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 97.0+ KB


# separando la col DTime en day, month, year

In [55]:
df_bike.head(2)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801


In [56]:
df_bike["dteday"].str.split(pat= "-", n = -1, expand = True)

,0,1,2
0,01,01,2018
1,02,01,2018
2,03,01,2018
3,04,01,2018
4,05,01,2018
...,...,...,...
725,27,12,2019
726,28,12,2019
727,29,12,2019
728,30,12,2019


In [57]:
df_bike["day"] = df_bike["dteday"].str.split(pat= "-", n = -1, expand = True)[0]
df_bike["month"] = df_bike["dteday"].str.split(pat= "-", n = -1, expand = True)[1]
df_bike["year"] = df_bike["dteday"].str.split(pat= "-", n = -1, expand = True)[2]

In [58]:
df_bike.head(2)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,day,month,year
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,01,01,2018
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,02,01,2018


In [62]:
df_bike["day"].dtypes

dtype('O')

In [52]:
# df_bike["dteday"] = pd.to_datetime(df_bike["dteday"])
# df_bike["day"] = pd.to_datetime(df_bike["day"])
# df_bike["month"] = pd.to_datetime(df_bike["month"])
# df_bike["year"] = pd.to_datetime(df_bike["year"])
# print(df_bike["dteday"].day)

# definir categoricas y numericas
# .reindex las columnas a gusto o uso
# .describe
# .stb.counts()
# .stb.missing()
# pairplot - Busca relaciones - ¿Hay que hacer algun ajuste?
# Boxplot
# Outliers - Identificarlos ¿Que hacer con ellos? %de cada columna
# Heatmap con la relacion
# ajuste lineal multiple
